# GBS Phylogeny of Eulychnia

Here we run a denovo assembly for an empirical GBS data set of the plant genus *Eulychnia* Phil. (Cactaceae) using the ipyrad Python API.

For this study we analyse 20 samples of *Eulychnia*, four of the sister genus *Austrocatuctus* Britton & Rose and one sample of *Corryocactus* Britton & Rose as outgroup. This data set is composed of double digested single-end ~100 bp reads from a GBS library prepared with the `PstI` and `MspI` cutting-enzymes. Its basically ddRAD, but people just call it differently, see [Campbell et al. 2018](https://doi.org/10.1111/2041-210X.13038). After the Assembly of the data set we make use of the ipyrad analysis tools to run downstream analyses in parallel. Unique to this data set is that we were able to incorporate material of all published taxonomic names (accepted names and synonyms) from their type locations. Addtionally, we also incorporated matierial of populations showing mophological differences.

GBS sequence reads are deposited at the European Nucleotide Archive (ENA)[https://www.ebi.ac.uk/ena/browser/home] under the accession PRJEB39114.

## Setup (software and parallelization)

To reproduce the study, start by installing ipyrad using conda (see ipyrad installation instructions [link](https://ipyrad.readthedocs.io/en/latest/3-installation.html) as well as the packages in the cell below. This is easiest to do in a terminal. Then open a jupyter-notebook, like this one, and follow along with the skript.

In [ ]:
## conda install ipyrad -c ipyrad
## conda install toytree -c eaton-lab
## conda install sra-tools -c bioconda
## conda install entrez-direct -c bioconda

In [1]:
## imports
import ipyrad as ip
import ipyrad.analysis as ipa
import ipyparallel as ipp
import toytree
import toyplot
print("ipyrad v. {}".format(ip.__version__))
print("toytree v. {}".format(toytree.__version__))

ipyrad v. 0.9.13
toytree v. 0.2.2


#### Parallel processes on independent Python kernels
To start a parallel client you must run the command-line program 'ipcluster'. This will essentially start a number of independent Python processes (kernels) which we can then send bits of work to do. The cluster can be stopped and restarted independently of this notebook, which is convenient for working on a cluster where connecting to many cores is not always immediately available.

Open a terminal and type the following command to start an ipcluster instance with N engines.

In [5]:
## ipcluster start --n=8

In [2]:
## connect to cluster 
ipyclient = ipp.Client()
#ipyclient.ids
print(ip.cluster_info(ipyclient))

Parallel connection | 
None


## Data Assembly
### Create an Assembly object and modify *ipyrad* params file
This object stores the parameters of the assembly and the organization of data files.

In [5]:
## you must provide a name for the Assembly
data = ip.Assembly("Eulychnia")

New Assembly: Eulychnia


Set parameters for the Assembly. This will raise an error if any of the parameter are not allowed beacuse they are the wrong type, or out of the allowed range.

In [8]:
## set parameters
data.set_params("project_dir", "Euly_Assembly")
data.set_params("sorted_fastq_path", "./fastq_Euly/*.fastq.gz")
data.set_params("clust_threshold", "0.85")
data.set_params("max_Hs_consens", (0.05))
data.set_params("restriction_overhang", ('TGCAG', 'GGCC'))
data.set_params("output_formats", "*")
data.set_params("datatype", "ddrad")

## see / print all parameters
data.get_params()

0   assembly_name               Eulychnia                                    
1   project_dir                 ./Euly_Assembly                              
2   raw_fastq_path                                                           
3   barcodes_path                                                            
4   sorted_fastq_path           ./fastq_Euly/*.fastq.gz                      
5   assembly_method             denovo                                       
6   reference_sequence                                                       
7   datatype                    ddrad                                        
8   restriction_overhang        ('TGCAG', 'GGCC')                            
9   max_low_qual_bases          5                                            
10  phred_Qscore_offset         33                                           
11  mindepth_statistical        6                                            
12  mindepth_majrule            6                               

### Assemble the data set

In [8]:
## run step 1 & 2 of the assembly
data.run("12", force = True)

Parallel connection | Atriplex: 8 cores
[####################] 100% 0:00:51 | loading reads        | s1 |
[####################] 100% 0:05:24 | processing reads     | s2 |


In [9]:
## run step 3-6 of the assembly
data.run("3456", force = True)

Parallel connection | Atriplex: 8 cores
[####################] 100% 0:00:00 | concatenating        | s3 |
[####################] 100% 0:00:38 | dereplicating        | s3 |
[####################] 100% 0:27:25 | clustering/mapping   | s3 |
[####################] 100% 0:00:08 | building clusters    | s3 |
[####################] 100% 0:00:01 | chunking clusters    | s3 |
[####################] 100% 0:58:05 | aligning clusters    | s3 |
[####################] 100% 0:00:13 | concat clusters      | s3 |
[####################] 100% 0:00:14 | calc cluster stats   | s3 |
[####################] 100% 0:01:56 | inferring [H, E]     | s4 |
[####################] 100% 0:00:13 | calculating depths   | s5 |
[####################] 100% 0:00:21 | chunking clusters    | s5 |
[####################] 100% 0:21:05 | consens calling      | s5 |
[####################] 100% 0:00:15 | indexing alleles     | s5 |
[####################] 100% 0:00:04 | concatenating inputs | s6 |
[####################] 100% 0:00:58 

### Final assembly step using different min_samples_locus settings
A single outgroup sample hat to be removed from the final assembly due to sequencing failure

In [7]:
## load assembly object
data = ip.load_json("Euly_Assembly/Eulychnia.json")

loading Assembly: Eulychnia
from saved path: ~/Documents/Eulychnia_ipyrad/Euly_Assembly/Eulychnia.json


In [8]:
## exclude samples from assembly with low read number
keep_list = [i for i in data.samples.keys() if i not in ["Aus_sp_ED3491"]]
data1 = data.branch("data1", subsamples = keep_list)

In [10]:
## assembly with 4 or more samples shared across all loci = 20 % | 80 % missing data
pops4 = data1.branch("pops4")
pops4.populations = {
    "ingroup": (4, [i for i in pops4.samples if "Eul_" in i]),
    "outgourp": (0, [i for i in pops4.samples if "Aus_" in i]),
    }
pops4.run("7", force = True)

## assembly with 6 or more samples shared across all loci = 30 % | 70 % missing data 
pops6 = data1.branch("pops6")
pops6.populations = {
    "ingroup": (6, [i for i in pops6.samples if "Eul_" in i]),
    "outgourp": (0, [i for i in pops6.samples if "Aus_" in i]),
    }
pops6.run("7", force = True)

## assembly with 8 or more samples shared across all loci = 40 % | 60 % missing data 
pops8 = data1.branch("pops8")
pops8.populations = {
    "ingroup": (8, [i for i in pops8.samples if "Eul_" in i]),
    "outgourp": (0, [i for i in pops8.samples if "Aus_" in i]),
    }
pops8.run("7", force = True)

## assembly with 10 or more samples shared across all loci = 50 % | 50 % missing data 
pops10 = data1.branch("pops10")
pops10.populations = {
    "ingroup": (10, [i for i in pops10.samples if "Eul_" in i]),
    "outgourp": (0, [i for i in pops10.samples if "Aus_" in i]),
    }
pops10.run("7", force = True)

Parallel connection | Atriplex: 8 cores
[####################] 100% 0:00:06 | applying filters     | s7 |
[####################] 100% 0:00:11 | building arrays      | s7 |
[####################] 100% 0:00:05 | writing conversions  | s7 |
[####################] 100% 0:00:59 | indexing vcf depths  | s7 |
[####################] 100% 0:00:26 | writing vcf output   | s7 |
Parallel connection | Atriplex: 8 cores
[####################] 100% 0:00:08 | applying filters     | s7 |
[####################] 100% 0:00:11 | building arrays      | s7 |
[####################] 100% 0:00:06 | writing conversions  | s7 |
[####################] 100% 0:00:54 | indexing vcf depths  | s7 |
[####################] 100% 0:00:26 | writing vcf output   | s7 |
Parallel connection | Atriplex: 8 cores
[####################] 100% 0:00:07 | applying filters     | s7 |
[####################] 100% 0:00:10 | building arrays      | s7 |
[####################] 100% 0:00:06 | writing conversions  | s7 |
[####################]

## Phylogenetic downstream analyses
First, access the data files of the assemblies and use them in phylogenetic downstream analyses. You might need to install additional packages which are not included in the ipyrad from the beginning as toytree/toyplot to plot phylogenies or RAxML or tetRAD. Use the following comands to install the packages in the terminal.

In [ ]:
# conda install raxml -c bioconda
# conda install toytree -c eaton-lab
# conda install tetrad -c eaton-lab -c conda-forge

In [3]:
## Load assemblies from their JSON file
pops4 = ip.load_json("Euly_Assembly/pops4.json")
pops6 = ip.load_json("Euly_Assembly/pops6.json")
pops8 = ip.load_json("Euly_Assembly/pops8.json")
pops10 = ip.load_json("Euly_Assembly/pops10.json")

loading Assembly: pops4
from saved path: ~/Documents/Eulychnia_ipyrad/Euly_Assembly/pops4.json
loading Assembly: pops6
from saved path: ~/Documents/Eulychnia_ipyrad/Euly_Assembly/pops6.json
loading Assembly: pops8
from saved path: ~/Documents/Eulychnia_ipyrad/Euly_Assembly/pops8.json
loading Assembly: pops10
from saved path: ~/Documents/Eulychnia_ipyrad/Euly_Assembly/pops10.json


### RAxML --- ML concatination tree inference
Here we are running phylegentic reconstructions on all assemblies using Maximum likelihood. In order to reduce code and to avoid tedious manully starting of every analysis by hand we gonna use a for loop to iterate through all assembly files. This way we can easily let it run and also make shure every analysis runs with identical settings. On a laptop with an i7-8550U and 16 GB RAM the analysis of a single assembly object takes ca. 6 hours (200 bootstraps).

In [4]:
## loop to run RAxML on all four assemblies (pops 4 -pops10)
for dset in [pops4, pops6, pops8, pops10]:
    rax = ipa.raxml(
        workdir = "./Euly_RAxML",
        name = dset.name,
        data = dset.outfiles.phy,
        N = 200,
        T = 8,
        o = ["Corr_Aus_Out_SFB635_ED4540"])
    rax.run(force = True)

job pops4 finished successfully
job pops6 finished successfully


KeyboardInterrupt: 

In [12]:
phy = "/home/tim/Documents/Eulychnia_ipyrad/Euly_Assembly/pops6_outfiles/pops6.phy"

In [15]:
## create a raxml analysis object for the pops6 data sets
rax = ipa.raxml(
    name = pops6.name,
    data = phy,
    workdir = "Euly_RAxML",
    T = 8,
    N = 200,
    o = ["Corr_Aus_Out_SFB635_ED4540"],
    )

## Print the raxml command
print(rax.command)

raxmlHPC-PTHREADS-SSE3 -f a -T 8 -m GTRGAMMA -n pops6 -w /home/tim/Documents/Eulychnia_ipyrad/Euly_RAxML -s /home/tim/Documents/Eulychnia_ipyrad/Euly_Assembly/pops6_outfiles/pops6.phy -p 54321 -N 200 -x 12345


In [16]:
## Run the analysis
rax.run(force = True)

job pops6 finished successfully


In [17]:
## access the resulting tree files
rax.trees

bestTree                   ~/Documents/Eulychnia_ipyrad/Euly_RAxML/RAxML_bestTree.pops6
bipartitions               ~/Documents/Eulychnia_ipyrad/Euly_RAxML/RAxML_bipartitions.pops6
bipartitionsBranchLabels   ~/Documents/Eulychnia_ipyrad/Euly_RAxML/RAxML_bipartitionsBranchLabels.pops6
bootstrap                  ~/Documents/Eulychnia_ipyrad/Euly_RAxML/RAxML_bootstrap.pops6
info                       ~/Documents/Eulychnia_ipyrad/Euly_RAxML/RAxML_info.pops6

In [24]:
## plot the resulting tree in the notebook with toytree
tre = toytree.tree(rax.trees.bipartitions)
#tre = toytree.tree("./analysis-raxml_09/RAxML_bipartitions.pops8")
rtre = tre.root(wildcard = "Corr")
# use canvas and axes function in order use export function
canvas, axes = rtre.ladderize(1).draw(
    width = 1100,
    height = 900,
    tip_labels_align = True,
    node_labels = rtre.get_node_values("support"),
    node_sizes=0,
    node_labels_style={"font-size": "12px",
                       "baseline-shift": "7px",
                       "-toyplot-anchor-shift": "-13px"},
    )

<svg class="toyplot-canvas-Canvas" height="900.0px" id="t4c972e4bb32b4ef884645dde6e163058" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 1100.0 900.0" width="1100.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> Corr_Aus_Out_SFB635_ED4540 Aus_spin_ED3494 Aus_cox_ED3492 Aus_ber_ED3490 Aus_sub_ED3493 Eul_rit_ED4430 Eul_rit_ED3489 Eul_sp_LaSerena_ED3112 Eul_cast_ED3109 Eul_cast_ED4667 Eul_aci_ED3110 Eul_hwank_Chorros_ED4234 Eul_chorro_procum_ED3113 Eul_vall_ED4802 Eul_elata_ED3114 Eul_brev_ED3111 Eul_PieCol_ED3115 Eul_tenuis_ED3116 Eul_barq_ED3117 Eul_saint-pieana_ED3118 Eul_tal_ED3119 Eul_iqui_pull_ED3120 Eul_iqui_morro_ED3121 Eul_iqu_aricensis_ED4541 Eul_iqui_ED3122 100 100 100 97 100 100 81 88 100 100 98 100 100 94 100 100 52 100 100 100 100 100 100

In [20]:
## exporting figures in SVG, PDF ...
import toyplot.svg
import toyplot.pdf
toyplot.svg.render(canvas, "Euly_RAxML/RAxML_figures/Euly_RAxML_pops6.svg")
toyplot.pdf.render(canvas, "Euly_RAxML/RAxML_figures/Euly_RAxML_pops6.pdf")

### tetrad --- quartet tree inference
The program tetrad, which applies the theory of phylogenetic invariants (*see Lake 1987*) to infer quartet trees based on a SNP alignment. It then uses the software wQMC to join the quartets into a species tree. This combined approach was first developed by *Chifman and Kubatko (2015)* in the software *SVDQuartets*.
[Link to the tutorial](https://nbviewer.jupyter.org/github/dereneaton/ipyrad/blob/master/tests/cookbook-tetrad.ipynb)

#### Setup `tetrad` tree inference
The first step is to create a named `tetrad` Class object, which requires a minimum of two aruments, a name and a sqeunce file. The sequence that you will typically want to enter is the `'.snps.phy'` file from `ipyrad`, which is a phylip formatted file with all SNPs. You can als pass it the `'.snps.map'` file, whic tell `tetrad` how the SNPs are linked with in loci, so that a single SNP can be randomly sampled in each bottstrap replicate.

In [25]:
tet = ipa.tetrad(
    name = 'EulyTet',
    workdir ="./Euly_Tetrad",
    data = "./Euly_Assembly/pops6_outfiles/pops6.snps.hdf5",
    mapfile = "./Euly_Assembly/pops6_outfiles/pops6.snpsmap",
    nboots = 500
    )

tet.run(ipyclient=ipyclient, force = True)

loading snps array [25 taxa x 115324 snps]
max unlinked SNPs per quartet (nloci): 16867
quartet sampler (random, nsamples**2.8): 8207 / 12650
Parallel connection | Atriplex: 8 cores
initializing quartet sets database
[####################] 100% 0:00:06 | inferring full tree    
[####################] 100% 0:00:01 | bootstrap inference 1 
[####################] 100% 0:00:02 | bootstrap inference 2 
[####################] 100% 0:00:01 | bootstrap inference 3 
[####################] 100% 0:00:01 | bootstrap inference 4 
[####################] 100% 0:00:02 | bootstrap inference 5 
[####################] 100% 0:00:02 | bootstrap inference 6 
[####################] 100% 0:00:01 | bootstrap inference 7 
[####################] 100% 0:00:02 | bootstrap inference 8 
[####################] 100% 0:00:02 | bootstrap inference 9 
[####################] 100% 0:00:02 | bootstrap inference 10 
[####################] 100% 0:00:02 | bootstrap inference 11 
[####################] 100% 0:00:02 | bootstrap 

[####################] 100% 0:00:02 | bootstrap inference 258 
[####################] 100% 0:00:02 | bootstrap inference 259 
[####################] 100% 0:00:02 | bootstrap inference 260 
[####################] 100% 0:00:02 | bootstrap inference 261 
[####################] 100% 0:00:02 | bootstrap inference 262 
[####################] 100% 0:00:02 | bootstrap inference 263 
[####################] 100% 0:00:02 | bootstrap inference 264 
[####################] 100% 0:00:02 | bootstrap inference 265 
[####################] 100% 0:00:02 | bootstrap inference 266 
[####################] 100% 0:00:02 | bootstrap inference 267 
[####################] 100% 0:00:02 | bootstrap inference 268 
[####################] 100% 0:00:02 | bootstrap inference 269 
[####################] 100% 0:00:02 | bootstrap inference 270 
[####################] 100% 0:00:02 | bootstrap inference 271 
[####################] 100% 0:00:02 | bootstrap inference 272 
[####################] 100% 0:00:02 | bootstrap inferen

In [26]:
## access the resulting tree files
tet.trees

('tree', '/home/tim/Documents/Eulychnia_ipyrad/Euly_Tetrad/EulyTet.tree')
('cons', '/home/tim/Documents/Eulychnia_ipyrad/Euly_Tetrad/EulyTet.tree.cons')
('boots', '/home/tim/Documents/Eulychnia_ipyrad/Euly_Tetrad/EulyTet.tree.boots')
('nhx', '/home/tim/Documents/Eulychnia_ipyrad/Euly_Tetrad/EulyTet.tree.nhx')

In [29]:
tre = toytree.tree("./Euly_Tetrad/EulyTet.tree.cons")
rtre = tre.root(wildcard = "Corr_Aus_")
#rtre.draw(tip_labels_align=True, node_labels="support")

# use canvas and axes function in order use export function
canvas, axes = rtre.ladderize(1).draw(
    width = 1400,
    height = 900,
    #use_edge_length = False,
    tip_labels_align = True,
    node_labels = rtre.get_node_values("support"),
    node_sizes=0,
    node_labels_style={"font-size": "16px",
                       "baseline-shift": "7px",
                       "-toyplot-anchor-shift": "-13px"},
    )

<svg class="toyplot-canvas-Canvas" height="900.0px" id="t19799b3efe0c49ee9093ff7526ac07b1" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 1400.0 900.0" width="1400.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> Corr_Aus_Out_SFB635_ED4540 Aus_spin_ED3494 Aus_cox_ED3492 Aus_sub_ED3493 Aus_ber_ED3490 Eul_rit_ED4430 Eul_rit_ED3489 Eul_sp_LaSerena_ED3112 Eul_cast_ED4667 Eul_cast_ED3109 Eul_aci_ED3110 Eul_hwank_Chorros_ED4234 Eul_elata_ED3114 Eul_vall_ED4802 Eul_chorro_procum_ED3113 Eul_brev_ED3111 Eul_PieCol_ED3115 Eul_tenuis_ED3116 Eul_saint-pieana_ED3118 Eul_barq_ED3117 Eul_tal_ED3119 Eul_iqui_pull_ED3120 Eul_iqui_morro_ED3121 Eul_iqu_aricensis_ED4541 Eul_iqui_ED3122 100 57 95 64 39 71 46 98 58 79 100 98 97 100 100 100 41 98 89 100 100 100 100

In [ ]:
## exporting figures in SVG, PDF ...
import toyplot.svg
import toyplot.pdf
toyplot.svg.render(canvas, "Euly_Tetrad/Tetrad_figures/Euly_Tetrad_pops6.svg")
toyplot.pdf.render(canvas, "Euly_Tetrad/Tetrad_figures/Euly_Tetrad_pops6.pdf")

#### Plotting ML and tet tree beside each other

In [41]:
## Load trees
Euly_ML_pops6 = toytree.tree("./Euly_RAxML/RAxML_bipartitions.pops6") 
Euly_Tet_pops6 = toytree.tree("./Euly_Tetrad/EulyTet.tree.cons")

## root the trees
rEuly_ML_pops6 = Euly_ML_pops6.root(wildcard = "Corr_Aus_")
rEuly_Tet_pops6 = Euly_Tet_pops6.root(wildcard = "Corr_Aus_")

# set dimensions of the canvas
canvas = toyplot.Canvas(width=900, height=600)

# dissect canvas into multiple cartesian areas (x1, x2, y1, y2)
ax0 = canvas.cartesian(bounds=('2%', '65%', '10%', '90%'))
ax1 = canvas.cartesian(bounds=('55%', '90%', '10%', '90%'))

# call draw with the 'axes' argument to pass it to a specific cartesian area
style = {
    "tip_labels_align": True,
    "tip_labels_style": {"font-size": "9px"},
    "node_labels_style": {"font-size": "12px",
                       "baseline-shift": "7px",
                       "-toyplot-anchor-shift": "-13px"},
}
rEuly_ML_pops6.ladderize(1).draw(
    axes=ax0, **style, node_sizes=0,
    node_labels = rEuly_ML_pops6.get_node_values("support"));
rEuly_Tet_pops6.ladderize(1).draw(
    axes=ax1, **style, node_sizes=0,
    node_labels = rEuly_Tet_pops6.get_node_values("support"));

# hide the axes (e.g, ticks and splines)
ax0.show=False; ax1.show=False

<svg class="toyplot-canvas-Canvas" height="600.0px" id="tfb6efa760ccd4a128a804eac06071913" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 900.0 600.0" width="900.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> Corr_Aus_Out_SFB635_ED4540 Aus_spin_ED3494 Aus_cox_ED3492 Aus_ber_ED3490 Aus_sub_ED3493 Eul_rit_ED4430 Eul_rit_ED3489 Eul_sp_LaSerena_ED3112 Eul_cast_ED3109 Eul_cast_ED4667 Eul_aci_ED3110 Eul_hwank_Chorros_ED4234 Eul_chorro_procum_ED3113 Eul_vall_ED4802 Eul_elata_ED3114 Eul_brev_ED3111 Eul_PieCol_ED3115 Eul_tenuis_ED3116 Eul_barq_ED3117 Eul_saint-pieana_ED3118 Eul_tal_ED3119 Eul_iqui_pull_ED3120 Eul_iqui_morro_ED3121 Eul_iqu_aricensis_ED4541 Eul_iqui_ED3122 100 100 100 97 100 100 81 88 100 100 98 100 100 94 100 100 52 100 100 100 100 100 100 Corr_Aus_Out_SFB635_ED4540 Aus_spin_ED3494 Aus_cox_ED3492 Aus_sub_ED3493 Aus_ber_ED3490 Eul_rit_ED4430 Eul_rit_ED3489 Eul_sp_LaSerena_ED3112 Eul_cast_ED4667 Eul_cast_ED3109 Eul_aci_ED3110 Eul_hwank_Chorros_ED4234 Eul_elata_ED3114 Eul_vall_ED4802 Eul_chorro_procum_ED3113 Eul_brev_ED3111 Eul_PieCol_ED3115 Eul_tenuis_ED3116 Eul_saint-pieana_ED3118 Eul_barq_ED3117 Eul_tal_ED3119 Eul_iqui_pull_ED3120 Eul_iqui_morro_ED3121 Eul_iqu_aricensis_ED4541 Eul_iqui_ED3122 100 57 95 64 39 71 46 98 58 79 100 98 97 100 100 100 41 98 89 100 100 100 100

In [42]:
## exporting figures in SVG, PDF ...
import toyplot.svg
import toyplot.pdf
toyplot.svg.render(canvas, "Euly_ML-Tetrad_pops6.svg")
toyplot.pdf.render(canvas, "Euly_ML-Tetrad_pops6.pdf")

In [3]:
## Load trees
Euly_ML_pops6 = toytree.tree("./Euly_RAxML/RAxML_bipartitions.pops6") 
Euly_ML_pops4 = toytree.tree("./Euly_RAxML/RAxML_bipartitions.pops4")

## root the trees
rEuly_ML_pops6 = Euly_ML_pops6.root(wildcard = "Corr_Aus_")
rEuly_ML_pops4 = Euly_ML_pops4.root(wildcard = "Corr_Aus_")

# set dimensions of the canvas
canvas = toyplot.Canvas(width=900, height=600)

# dissect canvas into multiple cartesian areas (x1, x2, y1, y2)
ax0 = canvas.cartesian(bounds=('2%', '65%', '10%', '90%'))
ax1 = canvas.cartesian(bounds=('55%', '90%', '10%', '90%'))

# call draw with the 'axes' argument to pass it to a specific cartesian area
style = {
    "tip_labels_align": True,
    "tip_labels_style": {"font-size": "9px"},
    "node_labels_style": {"font-size": "12px",
                       "baseline-shift": "7px",
                       "-toyplot-anchor-shift": "-13px"},
}
rEuly_ML_pops6.ladderize(1).draw(
    axes=ax0, **style, node_sizes=0,
    node_labels = rEuly_ML_pops6.get_node_values("support"));
rEuly_ML_pops4.ladderize(1).draw(
    axes=ax1, **style, node_sizes=0,
    node_labels = rEuly_ML_pops4.get_node_values("support"));

# hide the axes (e.g, ticks and splines)
ax0.show=False; ax1.show=False

<svg class="toyplot-canvas-Canvas" height="600.0px" id="t9a7d2a0d42084b5c853c612242b4ec3c" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 900.0 600.0" width="900.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> Corr_Aus_Out_SFB635_ED4540 Aus_spin_ED3494 Aus_cox_ED3492 Aus_ber_ED3490 Aus_sub_ED3493 Eul_rit_ED4430 Eul_rit_ED3489 Eul_sp_LaSerena_ED3112 Eul_cast_ED3109 Eul_cast_ED4667 Eul_aci_ED3110 Eul_hwank_Chorros_ED4234 Eul_chorro_procum_ED3113 Eul_vall_ED4802 Eul_elata_ED3114 Eul_brev_ED3111 Eul_PieCol_ED3115 Eul_tenuis_ED3116 Eul_barq_ED3117 Eul_saint-pieana_ED3118 Eul_tal_ED3119 Eul_iqui_pull_ED3120 Eul_iqui_morro_ED3121 Eul_iqu_aricensis_ED4541 Eul_iqui_ED3122 100 100 100 97 100 100 81 88 100 100 98 100 100 94 100 100 52 100 100 100 100 100 100 Corr_Aus_Out_SFB635_ED4540 Aus_spin_ED3494 Aus_cox_ED3492 Aus_ber_ED3490 Aus_sub_ED3493 Eul_rit_ED4430 Eul_rit_ED3489 Eul_sp_LaSerena_ED3112 Eul_cast_ED3109 Eul_cast_ED4667 Eul_aci_ED3110 Eul_hwank_Chorros_ED4234 Eul_chorro_procum_ED3113 Eul_vall_ED4802 Eul_elata_ED3114 Eul_brev_ED3111 Eul_PieCol_ED3115 Eul_tenuis_ED3116 Eul_saint-pieana_ED3118 Eul_barq_ED3117 Eul_tal_ED3119 Eul_iqui_pull_ED3120 Eul_iqui_morro_ED3121 Eul_iqui_ED3122 Eul_iqu_aricensis_ED4541 100 100 100 99 100 100 82 86 100 100 98 100 100 97 100 100 54 100 100 100 100 100 100

In [4]:
## exporting figures in SVG, PDF ...
import toyplot.svg
import toyplot.pdf
#toyplot.svg.render(canvas, "Euly_ML_pops6_&_4.svg")
toyplot.pdf.render(canvas, "Euly_ML_pops6_&_4.pdf")